<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>


# SD211 TP1

In [40]:
import numpy as np
from scipy import sparse
from scipy.optimize import check_grad
from scipy.sparse.linalg import svds
import math
from time import time
from scipy import optimize

In [41]:
def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool )
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask

In [42]:
rho=0.3
R,mask=load_movielens("ml-100k/u.data")
print(R.shape)

(943, 1682)


## Question 1.1
'minidata' échantillonne les premières 100 rangs et les premières 200 colonnes des données. 

In [43]:
counter=0
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if mask[i][j]==True:
            counter=counter+1
print(counter) #counter=100000

100000


## Question 1.2
Il y a 943 utilisateurs et 1682 films. Le nombre total de notes est 100000 

## Question 1.3
Si on pose |U|=1, |C|=1, R=10 et $\rho=2$, la fonction objective devient $g(x, y) = \frac{1}{2}(10-xy)^2 + (x^2+y^2)$<br>
On calcule ensuite la matrice hessienne:
$$\nabla^2 f=\begin{bmatrix}
 y^2+2& -10+2xy\\ 
 -10+2xy& x^2+2
\end{bmatrix}$$
Soit $h=(h_1,h_2)^T$ un vecteur non null.
$$h^T\nabla^2 fh=h_{1}^2(y^2+2)+h_{2}^2(x^2+2)+2h_{1}h_{2}(2xy-10)$$
Si on prend $h_1=h_2=x=y=1$, $h^T\nabla^2 fh=-10$, donc la matrice hessienne n'est pas positive, donc la fonction objective n'est pas convexe. 
Les deux gradients ne sont pas lipschitzien.
Par le même exemple, 
$$g(x,y) = \frac{1}{2}(10-xy)^2 + (x^2+y^2)$$
on a:
    $$\frac{\partial g}{\partial x} = (y^2+2)x - 10y$$
    $$\frac{\partial^2 g}{\partial x^2} = y^2 + 2$$
Quand $y \rightarrow \infty$, $\frac{\partial^2 g}{\partial x^2}\rightarrow \infty$, donc $\nabla_g(P)$ n'est pas lipschitzien. De la même façon, on peut conclure que $\nabla_g(Q)$ n'est pas lipschitzien.

## Question 2.1
Le gradient de g est $-{Q^0}^T(1_K\circ (R-Q^{0}P))+\rho P$ <br>
$\nabla ^2 g={Q^0}^{T}Q^0+\rho$ elle est définie positive car <br>
$h^T(Q^TQ)h=(Qh)^TQH=\left \| QH \right \|^2\geqslant 0$ et $\rho>0$ <br>
donc g est convexe

## Question 2.2

In [44]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) *mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = -Q0.T.dot(tmp)+rho*P

    return val, grad_P

def objective_Q(P0, Q, R, mask, rho):
   

    tmp = (R - Q.dot(P0)) *mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P0 ** 2))

    grad_Q = -tmp.dot(P0.T)+rho*Q

    return val, grad_Q

def func(P):
    P=P.reshape((4,1682))
    tmp = (R - Q0.dot(P)) *mask
    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))
    return val

def grad(P):
    P=P.reshape((4,1682))
    tmp = (R - Q0.dot(P)) *mask
    grad_P = -Q0.T.dot(tmp)+rho*P
    grad_P=grad_P.reshape(-1)
    return grad_P

Q0,S,P0=svds(R,k=4,return_singular_vectors=True)

In [45]:
t0=time()
print("la différence de gradient est %f" % check_grad(func,grad,P0.reshape(-1)))
print("ca fait %fs" % (time()-t0))

la différence de gradient est 1.156906
ca fait 153.864886s


## Question 2.3

In [46]:
def gradient(g, P0, gamma,epsilon):
    P=P0
    grad=g(P, Q0, R, mask, rho)[1]
    iteration=0
    while math.sqrt(np.sum(grad**2))>epsilon:
        P=P-grad*gamma
        val,grad=g(P, Q0, R, mask, rho)
        iteration=iteration+1
    return val,iteration

## Question 2.4
On prend $\frac{1}{L}$ comme le pas constant $\gamma$ où L est la constante lipschitzienne

In [47]:
L1=rho+math.sqrt(np.sum((Q0.T.dot(Q0))**2))
gamma1=1/L1
t0=time()
val,iteration=gradient(objective,P0,gamma1,1)
print("la valeur minimale est: %f" % val)
print("ca fait %fs" % (time()-t0))
print("Il y a %d itérations" % iteration)

la valeur minimale est: 369551.549915
ca fait 0.617983s
Il y a 26 itérations


## Question 3.1
Nous utilisons Armijo's line search

In [14]:
def armijo_line_search_P(g,P0,Q0,gamma0,epsilon):
    beta=0.5
    P=P0
    val,grad=g(P, Q0, R, mask, rho)
    sum_grad=np.sum(grad**2)
    gamma=gamma0
    iteration=0
    while math.sqrt(sum_grad)>epsilon:
        a=0.5
        b=2*gamma
        gamma=b
        _val=g(P-gamma*grad, Q0, R, mask, rho)[0]
        while _val>val-beta*gamma*sum_grad:
            iteration=iteration+1
            gamma=gamma*a
            _val=g(P-gamma*grad, Q0, R, mask, rho)[0]
        P=P-grad*gamma
        val,grad=g(P, Q0, R, mask, rho)
        sum_grad=np.sum(grad**2)
        iteration=iteration+1
    return val,iteration,P

def armijo_line_search_Q(g,P0,Q0,gamma0,epsilon):
    beta=0.5
    Q=Q0
    val,grad=g(P0, Q, R, mask, rho)
    sum_grad=np.sum(grad**2)
    gamma=gamma0
    iteration=0
    while math.sqrt(sum_grad)>epsilon:
        a=0.5
        b=2*gamma
        gamma=b
        _val=g(P0, Q-gamma*grad, R, mask, rho)[0]
        while _val>val-beta*gamma*sum_grad:
            iteration=iteration+1
            gamma=gamma*a
            _val=g(P0, Q-gamma*grad, R, mask, rho)[0]
        Q=Q-gamma*grad
        val,grad=g(P0, Q, R, mask, rho)
        sum_grad=np.sum(grad**2)
        iteration=iteration+1
    return val,iteration,Q

In [15]:
t0=time()
print("la valeur minimale est: %f" % armijo_line_search_P(objective,P0,Q0,0.5,1)[0])
print("ca fait %fs" % (time()-t0))
print("Il y a %d itérations" % armijo_line_search_P(objective,P0,Q0,0.5,1)[1])

la valeur minimale est: 369551.060504
ca fait 0.374842s
Il y a 13 itérations


## Question 3.2
Comme déjà calculé, $\nabla ^2 g={Q^0}^{T}Q^0+\rho$  est symétrique et définie positive, on peut utiliser la méthode des gradients conjugués

In [16]:
def grad_S(g,s):
    return (g(s+0.0001*s)-g(s))/0.0001
def armijo_line_search_S(g,s0,gamma0,epsilon):
    beta=0.5
    s=s0
    grad=grad_S(g,s)
    #print(grad)
    val=g(s)
    sum_grad=grad**2
    gamma=gamma0
    iteration=0
    while math.sqrt(sum_grad)>epsilon:
        a=0.5
        b=2*gamma
        gamma=b
        _val=g(s-gamma*grad)
        while _val>val-beta*gamma*sum_grad:
            iteration=iteration+1
            gamma=gamma*a
            _val=g(s-gamma*grad)
        s=s-grad*gamma
        val=g(s)
        grad=grad_S(g,s)
        sum_grad=grad**2
        #print(grad)
        iteration=iteration+1
    return val,iteration,s

def conjugue(g,P0,Q0,epsilon):
    P=P0
    grad=g(P,Q0,R,mask,rho)[1]
    d=-grad
    iteration=0
    #s=-1/(rho+math.sqrt(np.sum((Q0.T.dot(Q0))**2)))
    while(math.sqrt(np.sum(grad**2))>epsilon):
        f=lambda s:g(P+s*d,Q0,R,mask,rho)[0]
        #s=optimize.fmin(f,0.5)[0]
        s=armijo_line_search_S(f,1,0.5,100)[2]
        _P=P+s*d
        _grad=g(_P,Q0,R,mask,rho)[1]
        b=np.sum(_grad**2)/np.sum(grad**2)
        d=-_grad+b*d
        P=_P
        grad=_grad
        iteration=iteration+1
        #print(np.sum(grad**2))
    return g(P,Q0,R,mask,rho)[0],iteration

In [17]:
t0=time()
val,iteration=conjugue(objective,P0,Q0,1)
print("la valeur minimale est: %f" % val)
print("ca fait %fs" % (time()-t0))
print("Il y a %d itérations" % iteration)

la valeur minimale est: 369550.940809
ca fait 3.253746s
Il y a 7 itérations


## Question 3.3
<table>
<tr>
    <th></th>
    <th>gradient</th>
    <th>recherche linéaire</th>
    <th>conjugué</th>
</tr>
<tr>
    <td>résultat</td>
    <td>369551</td>
    <td>369551</td>
    <td>369550</td>
</tr>
<tr>
    <td>temps</td>
    <td>0.617983s</td>
    <td>0.374842s</td>
    <td>3.253746s</td>
</tr>
<tr>
    <td>itération</td>
    <td>26</td>
    <td>13</td>
    <td>7</td>
</tr>
</table>

On constate que les valeurs minimales obtenues sont presque les mêmes. Le méthode de recherche linéaire prend le moins de temps. Le méthode des gradients conjugués a le moins de nombre itérations, mais prend le plus de temps. 

## Question 4.1

In [18]:
def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = -Q.T.dot(tmp)+rho*P

    grad_Q = -tmp.dot(P.T)+rho*Q

    return val, grad_P, grad_Q


def total_objective_vectorized(PQvec, R, mask, rho):
    """
    Vectorisation de la fonction precedente de maniere a ne pas
    recoder la fonction gradient
    """

    # reconstruction de P et Q
    
    n_items = R.shape[1]
    n_users = R.shape[0]
    F = PQvec.shape[0] / (n_items + n_users)
    F=int(F)
    Pvec = PQvec[0:n_items*F]
    Qvec = PQvec[n_items*F:]
    P = np.reshape(Pvec, (F, n_items))
    Q = np.reshape(Qvec, (n_users, F))
    
    val, grad_P, grad_Q = total_objective(P, Q, R, mask, rho)
    return val, np.concatenate([grad_P.ravel(), grad_Q.ravel()])

def armijo_line_search_total(g,PQ0,gamma0,epsilon):
    beta=0.5
    PQ=PQ0
    val,grad=g(PQ,R,mask,rho)
    gamma=gamma0
    iteration=0
    sum_grad=np.sum(grad**2)
    while math.sqrt(sum_grad)>epsilon:
        a=0.5
        b=2*gamma
        gamma=b
        _val=g(PQ-gamma*grad,R, mask, rho)[0]
        while _val>val-gamma*beta*sum_grad:
            iteration=iteration+1
            gamma=gamma*a
            _val=g(PQ-gamma*grad,R, mask, rho)[0]
        PQ=PQ-gamma*grad
        val,grad=g(PQ,R,mask,rho)
        sum_grad=np.sum(grad**2)
        iteration=iteration+1
    return val,iteration,PQ

In [19]:
PQ=np.concatenate([P0.ravel(), Q0.ravel()])
t0=time()
val,iteration,PQ=armijo_line_search_total(total_objective_vectorized,PQ,0.1,100)
print("la valeur minimale est: %f" % val)
print("ca fait %fs" % (time()-t0))
print("Il y a %d itérations" % iteration)

la valeur minimale est: 36035.485912
ca fait 10.721682s
Il y a 280 itérations


Comme la fonction objective n'est pas convexe, la valeur obtenue est un minimum local mais pas un minimum global.

## Question 4.2

Quand on fixe P ou Q, la fonction objective est convexe et donc sa valeur diminue après chaque itération.Et puis, comme la fonction objective est positive donc elle est bornée. Donc cette méthode converge.

## Question 4.3

In [29]:
def moindres_carres_alternes(P0,Q0):
    P=P0
    Q=Q0
    iteration=0
    while True:
        L1=rho+math.sqrt(np.sum((Q.T.dot(Q))**2))  
        gamma1=1/L1
        _P=armijo_line_search_P(objective,P,Q,gamma1,100)[2]
        L2=rho+math.sqrt(np.sum((P.dot(P.T))**2))
        gamma2=1/L2
        _Q=armijo_line_search_Q(objective_Q,_P,Q,gamma2,100)[2]
        iteration=iteration+1
        diff=total_objective(P, Q, R, mask, rho)[0]-total_objective(_P, _Q, R, mask, rho)[0]
        diff=abs(diff)
        if (diff/total_objective(P, Q, R, mask, rho)[0]<0.01):
            break
        else:
            P=_P
            Q=_Q
        #print(total_objective(P, Q, R, mask, rho)[0])
    return total_objective(_P, _Q, R, mask, rho)[0],_P,_Q,iteration

In [33]:
t0=time()
val,P,Q,iteration=moindres_carres_alternes(P0,Q0)
print(time()-t0)

349.5847442150116


In [36]:
print("la valeur minimale est: %f" % val)
print("ca fait %fs" % 349.5847442150116)

la valeur minimale est: 57896.225177
ca fait 349.584744s


## Question 4.4 
Les résultats obtenus par ces deux méthodes ne sont pas la solution optimale mais des minimums locals. Leurs P, Q, $\hat{R}$ obtenus ne sont pas le même. La méthode des moindres carrés alternés prend beaucoup plus de temps.

<table>
<tr>
    <th></th>
    <th>gradient avec recherche linéaire</th>
    <th>moindres carrés alternés</th>
</tr>
<tr>
    <td>résultat</td>
    <td>36035.485912</td>
    <td>57896.225177</td>
<tr>
<tr>
    <td>temps</td>
    <td>10.721682s</td>
    <td>349.584744s</td>
<tr>
</table>

## Question 4.5 
Comme la méthode du gradient avec recherche linéaire a une meilleur performance, on utilise son résultat. Et puis, on recommande des films que l'utilisateur n'a pas vus. 

In [37]:
PQ=np.concatenate([P0.ravel(), Q0.ravel()])
val,iteration,PQ=armijo_line_search_total(total_objective_vectorized,PQ,0.1,100)

In [38]:
n_items = R.shape[1]
n_users = R.shape[0]
F = PQ.shape[0] / (n_items + n_users)
F=int(F)
Pvec = PQ[0:n_items*F]
Qvec = PQ[n_items*F:]
P = np.reshape(Pvec, (F, n_items))
Q = np.reshape(Qvec, (n_users, F))
_R=Q.dot(P)
score = _R[300, :].ravel()
_mask=mask[300, :].ravel()
rank = (-score).argsort()
print("dix premiers films recommandés: ")
counter=0
i=0
while True:
    if(_mask[rank[i]]!=0):
        counter=counter+1
        print("%d: movie %d, score %f" % (counter, rank[i], score[rank[i]]))
        if(counter==10):
            break
    i=i+1

dix premiers films recommandés: 
1: movie 49, score 4.646734
2: movie 63, score 4.600738
3: movie 173, score 4.532604
4: movie 21, score 4.432318
5: movie 95, score 4.431730
6: movie 180, score 4.426008
7: movie 171, score 4.396582
8: movie 11, score 4.383966
9: movie 317, score 4.355034
10: movie 172, score 4.303764
